In [2]:
%load_ext autoreload
%autoreload 2
from main  import *

The autoreload extension is already loaded. To reload it, use:

  %reload_ext

autoreload

In [3]:
opt = Config()    
opt.caption_data_path = 'caption.pth' # 原始数据
opt.test_img = 'img/example.jpeg' # 输入图片
opt.use_gpu = False  # 是否使用GPU(没必要)
opt.model_ckpt='caption_0914_1947' # 预训练的模型

In [4]:
# 数据预处理
data = t.load(opt.caption_data_path)
word2ix,ix2word = data['word2ix'],data['ix2word']

IMAGENET_MEAN =  [0.485, 0.456, 0.406]
IMAGENET_STD =  [0.229, 0.224, 0.225]
normalize =  tv.transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
transforms = tv.transforms.Compose([
            tv.transforms.Scale(opt.scale_size),
            tv.transforms.CenterCrop(opt.img_size),
            tv.transforms.ToTensor(),
            normalize
    ])
img_ = Image.open(opt.test_img)
img = transforms(img_).unsqueeze(0)
img_.resize((int(img_.width*256/img_.height),256))

D:\ProgramData\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
# 用resnet50来提取图片特征
# 如果本地没有预训练的模型文件，会自动下载
resnet50 = tv.models.resnet50(True).eval()
del resnet50.fc
resnet50.fc = lambda x:x
if opt.use_gpu:
    resnet50.cuda() 
    img = img.cuda()
img_feats = resnet50(Variable(img,volatile=True))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\Jizong/.torch\models\resnet50-19c8e357.pth


  0%|          | 0/102502400 [00:00<?, ?it/s]

  0%|          | 57344/102502400 [00:00<04:40, 365437.49it/s]

  0%|          | 106496/102502400 [00:00<04:20, 393743.58it/s]

  0%|          | 180224/102502400 [00:00<03:44, 455375.08it/s]

...

 99%|█████████▉| 101810176/102502400 [02:05<00:00, 1723496.14it/s]

100%|█████████▉| 102055936/102502400 [02:05<00:00, 1874613.87it/s]

100%|█████████▉| 102252544/102502400 [02:05<00:00, 1893595.10it/s]

100%|█████████▉| 102465536/102502400 [02:05<00:00, 1946595.57it/s]

100%|██████████| 102502400/102502400 [02:05<00:00, 815200.61it/s] 

In [6]:
# Caption模型
model = CaptionModel(opt,word2ix,ix2word)
model = model.load(opt.model_ckpt).eval()
if opt.use_gpu:
     model.cuda()

results = model.generate(img_feats.data[0])
print('\r\n'.join(results))

RuntimeError: size mismatch, m1: [1 x 8192], m2: [2048 x 256] at c:\anaconda2\conda-bld\pytorch_1519496000060\work\torch\lib\th\generic/THTensorMath.c:1434